In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [3]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [4]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [5]:
def Tree_Reg(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe UV Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
    y = df['Peak_eV']
    z = df['FWHM_eV']
    group = df['Expt. No']

    regressor_peak = DecisionTreeRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                              random_state=0)
    regressor_fwhm = DecisionTreeRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                              random_state=0)
    regressor_cascade = DecisionTreeRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                              random_state=0)

    gkf = GroupKFold(n_splits=10)
    matrix = np.zeros([10,13])
    matrix_fwhm = np.zeros([10,13])
    matrix_cascade = np.zeros([10,14])
    for i in range(0,1):
        j = 0
        for train_index, test_index in gkf.split(X, groups=group):
            #Separate out training and test sets
            X_train = X.iloc[train_index, :]
            X2_train = X2.iloc[train_index, :]
            y_train = y[train_index]
            z_train = z[train_index]
            X_test = X.iloc[test_index, :]
            y_test = y[test_index]
            z_test = z[test_index]

            #prediction for Peak
            regressor_peak.fit(X_train, y_train)
            y_pred = regressor_peak.predict(X_test)
            #prediction for FWHM (direct)
            regressor_fwhm.fit(X_train, z_train)
            y_pred_fwhm = regressor_fwhm.predict(X_test)
            #prediction for FWHM (cascade)
            regressor_cascade.fit(X2_train, z_train)
            X_test.insert(5, "Peak_eV", regressor_peak.predict(X_test), True)
            y_pred_cascade = regressor_cascade.predict(X_test)

            #errors in eV
            matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
            matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
            matrix[10*i + j,2] = rae(y_test, y_pred)
            matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

            matrix_fwhm[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,1] = metrics.r2_score(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,2] = rae(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))

            matrix_cascade[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,1] = metrics.r2_score(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,2] = rae(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))

            #errors in nm
            #UNnormalize the data first
            y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            z_test = z_test*(0.35)+0.05
            y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
            y_pred_cascade = y_pred_cascade*(0.35)+0.05
            matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
            matrix[10*i + j,8:13] = regressor_peak.feature_importances_

            matrix_fwhm[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
            matrix_fwhm[10*i + j,8:13] = regressor_fwhm.feature_importances_

            matrix_cascade[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
            matrix_cascade[10*i + j,8:14] = regressor_cascade.feature_importances_

            j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = matrix.mean(0)[:8]
    matrix_2[0,8:16] = np.std(matrix[:,:8], axis=0)
    matrix_2[0,16:21] = matrix.mean(0)[8:13]

    matrix_3 = np.zeros([1,21])
    matrix_3[0,:8] = matrix_fwhm.mean(0)[:8]
    matrix_3[0,8:16] = np.std(matrix_fwhm[:,:8], axis=0)
    matrix_3[0,16:21] = matrix_fwhm.mean(0)[8:13]

    matrix_4 = np.zeros([1,22])
    matrix_4[0,:8] = matrix_cascade.mean(0)[:8]
    matrix_4[0,8:16] = np.std(matrix_cascade[:,:8], axis=0)
    matrix_4[0,16:22] = matrix_cascade.mean(0)[8:14]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0], matrix_2[0,8])
    print("R2 (norm, eV):", matrix_2[0,1], matrix_2[0,9])
    print("RAE (norm, eV):", matrix_2[0,2], matrix_2[0,10])
    print("RMSE (norm, eV):", matrix_2[0,3], matrix_2[0,11])
    print("Importances", matrix_2[0,8:13])
    print("MAE (nm):", matrix_2[0,4], matrix_2[0,12])
    print("R2 (nm):", matrix_2[0,5], matrix_2[0,13])
    print("RAE (nm):", matrix_2[0,6], matrix_2[0,14])
    print("RMSE (nm):", matrix_2[0,7], matrix_2[0,15])

    print('\033[1m' + 'Absorption FWHM (direct)' + '\033[0m')
    print("MAE (norm, eV):", matrix_3[0,0], matrix_3[0,8])
    print("R2 (norm, eV):", matrix_3[0,1], matrix_3[0,9])
    print("RAE (norm, eV):", matrix_3[0,2], matrix_3[0,10])
    print("RMSE (norm, eV):", matrix_3[0,3], matrix_3[0,11])
    print("Importances", matrix_3[0,8:13])
    print("MAE (nm):", matrix_3[0,4], matrix_3[0,12])
    print("R2 (nm):", matrix_3[0,5], matrix_3[0,13])
    print("RAE (nm):", matrix_3[0,6], matrix_3[0,14])
    print("RMSE (nm):", matrix_3[0,7], matrix_3[0,15])

    print('\033[1m' + 'Absorption FWHM (cascade)' + '\033[0m')
    print("MAE (norm, eV):", matrix_4[0,0], matrix_4[0,8])
    print("R2 (norm, eV):", matrix_4[0,1], matrix_4[0,9])
    print("RAE (norm, eV):", matrix_4[0,2], matrix_4[0,10])
    print("RMSE (norm, eV):", matrix_4[0,3], matrix_4[0,11])
    print("Importances", matrix_4[0,8:14])
    print("MAE (nm):", matrix_4[0,4], matrix_4[0,12])
    print("R2 (nm):", matrix_4[0,5], matrix_4[0,13])
    print("RAE (nm):", matrix_4[0,6], matrix_4[0,14])
    print("RMSE (nm):", matrix_4[0,7], matrix_4[0,15])

    return {'loss': matrix_2[0,3] ** 2, 'status': STATUS_OK }

In [6]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = { 'x1': hp.quniform('x1', 10,20,1),
            'x2': hp.quniform('x2', 2,20,1),
            'x3': hp.quniform('x3', 1,10,1),
            'x4': hp.quniform('x4', 1,5,1),
        }

best = fmin(Tree_Reg,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.059977029716876004                                                                                                   
0.002784357076303037                                                                                                   
R2 (norm, eV):                                                                                                         
0.7967260390247853                                                                                                     
0.03223424449339665                                                                                                    
RAE (norm, eV):                                                                                                        
0.4248401763419995                              

[0.00294161 0.09780948 0.04002462 0.00755485 0.29163767 0.10407635]                                                    
MAE (nm):                                                                                                              
3.0577438702534954                                                                                                     
0.29163767024117576                                                                                                    
R2 (nm):                                                                                                               
0.5283360975386173                                                                                                     
0.10407635336560556                                                                                                    
RAE (nm):                                                                                                              
0.5749808055219477                      

MAE (norm, eV):                                                                                                        
0.0344573553891826                                                                                                     
0.00401051034910699                                                                                                    
R2 (norm, eV):                                                                                                         
0.4872473372377623                                                                                                     
0.13825290990085987                                                                                                    
RAE (norm, eV):                                                                                                        
0.6049593547106419                                                                                                     
0.07156747905219929                     

MAE (nm):                                                                                                              
3.5115220384683155                                                                                                     
0.24866203071927015                                                                                                    
R2 (nm):                                                                                                               
0.40939428525018284                                                                                                    
0.09107448790912583                                                                                                    
RAE (nm):                                                                                                              
0.6607757088557025                                                                                                     
0.05854794842646862                     

0.030515644216549852                                                                                                   
0.0023759782927683837                                                                                                  
R2 (norm, eV):                                                                                                         
0.5966474485883408                                                                                                     
0.06366362088027114                                                                                                    
RAE (norm, eV):                                                                                                        
0.5359737620747109                                                                                                     
0.04555292645958774                                                                                                    
RMSE (norm, eV):                        

...

0.06606055685303523                                                                                                    
RMSE (norm, eV):                                                                                                       
0.04715313387996543                                                                                                    
0.008215873646516807                                                                                                   
Importances                                                                                                            
[0.00346294 0.12887355 0.06606056 0.00821587 0.31718711]                                                               
MAE (nm):                                                                                                              
3.1934585474782335                                                                                                     
0.3171871116537635                      

0.04704525959259387                                                                                                    
RMSE (nm):                                                                                                             
16.47281774558906                                                                                                      
1.499767840082025                                                                                                      
Absorption FWHM (direct)                                                                                       
MAE (norm, eV):                                                                                                        
0.0300396696821423                                                                                                     
0.002255815341684414                                                                                                   
R2 (norm, eV):                                  

RMSE (norm, eV):                                                                                                       
0.07840193308311358                                                                                                    
0.007707151773000278                                                                                                   
Importances                                                                                                            
[0.00495864 0.04576934 0.04447316 0.00770715 0.99583046]                                                               
MAE (nm):                                                                                                              
12.822525793796535                                                                                                     
0.9958304644594279                                                                                                     
R2 (nm):                                

RMSE (nm):                                                                                                             
4.49770581726432                                                                                                       
0.5735549141228772                                                                                                     
Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.05905801141500908                                                                                                    
0.004227206311351793                                                                                                   
R2 (norm, eV):                                                                                                         
0.7991223968031349                              

0.04732438589808855                                                                                                    
0.00828928180911983                                                                                                    
Importances                                                                                                            
[0.00332729 0.1153457  0.05072059 0.00828928 0.22768034 0.14602629]                                                    
MAE (nm):                                                                                                              
3.154345557191969                                                                                                      
0.22768033808716578                                                                                                    
R2 (nm):                                                                                                               
0.48143989038378265                     